# 01. JRA Horse Racing Data Exploration

This notebook explores the Kaggle JRA Horse Racing Dataset to understand:
- Data structure and available features
- Data quality and missing values
- Key statistics for horse racing prediction
- Feature engineering opportunities

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.family'] = 'DejaVu Sans'

# Set display options
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 200)

In [ ]:
# Data paths
RAW_DATA_DIR = Path('../data/raw')

# List all CSV files
csv_files = list(RAW_DATA_DIR.glob('*.csv'))
print(f"Found {len(csv_files)} CSV files:")
for f in csv_files:
    size_mb = f.stat().st_size / (1024 * 1024)
    print(f"  - {f.name} ({size_mb:.1f} MB)")

## 1. Load and Inspect Data

In [ ]:
# Load all CSV files into a dictionary
dfs = {}
for f in csv_files:
    name = f.stem
    print(f"Loading {name}...")
    dfs[name] = pd.read_csv(f)
    print(f"  Shape: {dfs[name].shape}")
    print(f"  Columns: {list(dfs[name].columns)[:10]}..." if len(dfs[name].columns) > 10 else f"  Columns: {list(dfs[name].columns)}")
    print()

In [ ]:
# Display detailed info for each dataframe
for name, df in dfs.items():
    print(f"\n{'='*60}")
    print(f"DataFrame: {name}")
    print(f"{'='*60}")
    print(f"Shape: {df.shape}")
    print(f"\nColumn types:")
    print(df.dtypes)
    print(f"\nFirst 3 rows:")
    display(df.head(3))

## 2. Data Quality Check

In [ ]:
# Check missing values for each dataframe
for name, df in dfs.items():
    print(f"\n{'='*60}")
    print(f"Missing values: {name}")
    print(f"{'='*60}")
    
    missing = df.isnull().sum()
    missing_pct = (missing / len(df) * 100).round(2)
    missing_df = pd.DataFrame({
        'Missing': missing,
        'Percentage': missing_pct
    })
    missing_df = missing_df[missing_df['Missing'] > 0].sort_values('Missing', ascending=False)
    
    if len(missing_df) > 0:
        print(missing_df)
    else:
        print("No missing values!")

## 3. Race Data Analysis

Assuming there's a main race results table. Adjust column names based on actual data.

In [ ]:
# Identify the main results dataframe (likely the largest or named 'results', 'race', etc.)
# This will need adjustment based on actual file names
main_df_name = None
for name in dfs.keys():
    if 'result' in name.lower() or 'race' in name.lower():
        main_df_name = name
        break

if main_df_name is None and len(dfs) > 0:
    # Use the largest dataframe as main
    main_df_name = max(dfs.keys(), key=lambda x: len(dfs[x]))

if main_df_name:
    print(f"Using '{main_df_name}' as main dataframe")
    df = dfs[main_df_name].copy()
else:
    print("No data loaded. Please download the dataset first.")

In [ ]:
# Display all columns
if 'df' in dir():
    print("All columns:")
    for i, col in enumerate(df.columns):
        print(f"{i+1:3}. {col}")

## 4. Filter Data for 2019-2021

In [ ]:
# Look for date column and filter to 2019-2021
# Column names will vary - common patterns: 'date', 'race_date', 'year', etc.
if 'df' in dir():
    date_cols = [col for col in df.columns if any(x in col.lower() for x in ['date', 'year', '日付', '年'])]
    print(f"Potential date columns: {date_cols}")
    
    # Try to parse and filter by year
    for col in date_cols:
        try:
            if df[col].dtype == 'object':
                df['_year'] = pd.to_datetime(df[col], errors='coerce').dt.year
            else:
                df['_year'] = df[col]
            
            year_counts = df['_year'].value_counts().sort_index()
            print(f"\nYear distribution from '{col}':")
            print(year_counts)
            
            # Filter to 2019-2021
            df_filtered = df[(df['_year'] >= 2019) & (df['_year'] <= 2021)].copy()
            print(f"\nFiltered to 2019-2021: {len(df_filtered)} rows")
            df = df_filtered
            break
        except:
            continue

## 5. Key Statistics for Horse Racing

In [ ]:
# This section will analyze race-specific statistics
# Adjust column names based on actual data

if 'df' in dir():
    # Try to identify key columns
    print("Sample of data:")
    display(df.head(10))
    
    # Basic statistics
    print("\nNumerical statistics:")
    display(df.describe())

In [ ]:
# Identify potential feature columns based on naming patterns
if 'df' in dir():
    col_categories = {
        'horse': ['horse', '馬', 'uma'],
        'jockey': ['jockey', '騎手', 'kishu'],
        'trainer': ['trainer', '調教', 'choukyou'],
        'odds': ['odds', 'オッズ', '単勝', '複勝'],
        'position': ['position', '着順', 'rank', 'order', '順位'],
        'weight': ['weight', '体重', '斤量', 'kinryou'],
        'distance': ['distance', '距離', 'kyori'],
        'track': ['track', 'course', 'コース', '馬場', 'baba'],
        'corner': ['corner', 'コーナー', '通過'],
        'time': ['time', 'タイム', '秒', 'sec'],
        'blood': ['sire', 'father', '父', 'dam', 'mother', '母'],
    }
    
    print("Column categorization:")
    for category, patterns in col_categories.items():
        matching_cols = [col for col in df.columns if any(p in col.lower() for p in patterns)]
        if matching_cols:
            print(f"\n{category.upper()}:")
            for col in matching_cols:
                print(f"  - {col}")

## 6. Finishing Position Distribution

In [ ]:
# Analyze finishing positions (target variable for prediction)
if 'df' in dir():
    # Try to find finishing position column
    position_cols = [col for col in df.columns if any(x in col.lower() for x in ['着順', 'rank', 'position', 'order', 'finish'])]
    
    if position_cols:
        pos_col = position_cols[0]
        print(f"Using '{pos_col}' as finishing position")
        
        # Clean and analyze
        df[pos_col] = pd.to_numeric(df[pos_col], errors='coerce')
        
        plt.figure(figsize=(12, 5))
        
        plt.subplot(1, 2, 1)
        df[pos_col].value_counts().sort_index().head(18).plot(kind='bar')
        plt.title('Finishing Position Distribution')
        plt.xlabel('Position')
        plt.ylabel('Count')
        
        plt.subplot(1, 2, 2)
        df[pos_col].value_counts().sort_index().head(18).cumsum().div(len(df)).mul(100).plot(kind='line', marker='o')
        plt.title('Cumulative % by Position')
        plt.xlabel('Position')
        plt.ylabel('Cumulative %')
        
        plt.tight_layout()
        plt.show()

## 7. Odds Analysis

In [ ]:
# Analyze odds distribution and relationship with results
if 'df' in dir():
    odds_cols = [col for col in df.columns if any(x in col.lower() for x in ['odds', 'オッズ', '単勝', 'tansho'])]
    
    if odds_cols and position_cols:
        odds_col = odds_cols[0]
        pos_col = position_cols[0]
        print(f"Odds column: {odds_col}")
        
        df[odds_col] = pd.to_numeric(df[odds_col], errors='coerce')
        
        plt.figure(figsize=(12, 5))
        
        # Odds distribution for winners vs all
        plt.subplot(1, 2, 1)
        df[df[pos_col] == 1][odds_col].clip(upper=100).hist(bins=50, alpha=0.7, label='Winners')
        df[odds_col].clip(upper=100).hist(bins=50, alpha=0.3, label='All')
        plt.title('Odds Distribution')
        plt.xlabel('Odds')
        plt.legend()
        
        # Win rate by odds range
        plt.subplot(1, 2, 2)
        df['odds_bin'] = pd.cut(df[odds_col], bins=[0, 2, 5, 10, 20, 50, 100, 1000])
        win_rate = df.groupby('odds_bin')[pos_col].apply(lambda x: (x == 1).mean() * 100)
        win_rate.plot(kind='bar')
        plt.title('Win Rate by Odds Range')
        plt.xlabel('Odds Range')
        plt.ylabel('Win Rate (%)')
        plt.xticks(rotation=45)
        
        plt.tight_layout()
        plt.show()

## 8. Umatan (Exacta) Analysis

For Umatan, we need to predict 1st and 2nd place in exact order.

In [ ]:
# Analyze 1st-2nd combinations
if 'df' in dir():
    # Find race ID column
    race_id_cols = [col for col in df.columns if any(x in col.lower() for x in ['race_id', 'レースid', 'race_no'])]
    
    if race_id_cols and position_cols:
        race_col = race_id_cols[0] if race_id_cols else None
        pos_col = position_cols[0]
        
        if race_col:
            print(f"Race column: {race_col}")
            
            # Count horses per race
            horses_per_race = df.groupby(race_col).size()
            print(f"\nHorses per race:")
            print(f"  Min: {horses_per_race.min()}")
            print(f"  Max: {horses_per_race.max()}")
            print(f"  Mean: {horses_per_race.mean():.1f}")
            print(f"  Median: {horses_per_race.median():.1f}")
            
            plt.figure(figsize=(10, 4))
            horses_per_race.value_counts().sort_index().plot(kind='bar')
            plt.title('Distribution of Horses per Race')
            plt.xlabel('Number of Horses')
            plt.ylabel('Number of Races')
            plt.show()

## 9. Track and Distance Analysis

In [ ]:
# Analyze by track surface and distance
if 'df' in dir():
    distance_cols = [col for col in df.columns if any(x in col.lower() for x in ['distance', '距離', 'kyori'])]
    track_cols = [col for col in df.columns if any(x in col.lower() for x in ['芝', 'turf', 'dirt', 'ダート', 'surface', '馬場'])]
    
    if distance_cols:
        dist_col = distance_cols[0]
        df[dist_col] = pd.to_numeric(df[dist_col], errors='coerce')
        
        plt.figure(figsize=(12, 4))
        
        plt.subplot(1, 2, 1)
        df[dist_col].value_counts().sort_index().plot(kind='bar')
        plt.title('Race Distance Distribution')
        plt.xlabel('Distance (m)')
        plt.ylabel('Count')
        
        if track_cols:
            track_col = track_cols[0]
            plt.subplot(1, 2, 2)
            df[track_col].value_counts().plot(kind='bar')
            plt.title('Track Surface Distribution')
            plt.xlabel('Surface')
            plt.ylabel('Count')
        
        plt.tight_layout()
        plt.show()

## 10. Racecourse Analysis

In [ ]:
# JRA Racecourses
RACECOURSE_CODES = {
    1: "札幌", 2: "函館", 3: "福島", 4: "新潟", 5: "東京",
    6: "中山", 7: "中京", 8: "京都", 9: "阪神", 10: "小倉"
}

if 'df' in dir():
    course_cols = [col for col in df.columns if any(x in col.lower() for x in ['場', 'course', 'racecourse', 'venue', 'place'])]
    
    if course_cols:
        course_col = course_cols[0]
        print(f"Racecourse column: {course_col}")
        print(f"\nRaces by racecourse:")
        
        course_counts = df[course_col].value_counts()
        print(course_counts)
        
        plt.figure(figsize=(10, 5))
        course_counts.plot(kind='bar')
        plt.title('Races by Racecourse')
        plt.xlabel('Racecourse')
        plt.ylabel('Number of Runners')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

## 11. Feature Engineering Ideas

Based on the data exploration, here are potential features to engineer:

In [ ]:
# Summary of data structure for feature engineering
if 'df' in dir():
    print("="*60)
    print("DATA SUMMARY FOR FEATURE ENGINEERING")
    print("="*60)
    print(f"\nTotal rows: {len(df):,}")
    print(f"Total columns: {len(df.columns)}")
    print(f"\nColumn types:")
    print(df.dtypes.value_counts())
    
    print("\n" + "="*60)
    print("RECOMMENDED FEATURES TO ENGINEER")
    print("="*60)
    
    features = [
        "1. Running Style (脚質) - from corner passing positions",
        "2. Jockey/Trainer Win Rates - rolling calculations",
        "3. Distance Aptitude - past performance by distance category",
        "4. Surface Aptitude - turf vs dirt performance",
        "5. Track Aptitude - performance at specific racecourses",
        "6. Recent Form - last N race results with time decay",
        "7. Class Performance - results by race grade",
        "8. Weight Change - horse weight difference from last race",
        "9. Post Position Stats - position-specific win rates",
        "10. Pace Analysis - early/mid/late position changes",
    ]
    
    for f in features:
        print(f)

## 12. Save Processed Data Info

In [ ]:
# Save column mapping for feature engineering
if 'df' in dir():
    column_info = {
        'all_columns': list(df.columns),
        'dtypes': df.dtypes.to_dict(),
        'shape': df.shape,
        'missing': df.isnull().sum().to_dict()
    }
    
    # Save as JSON for reference
    import json
    output_path = Path('../data/processed/column_info.json')
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump({
            'columns': column_info['all_columns'],
            'shape': list(column_info['shape']),
        }, f, indent=2, ensure_ascii=False)
    
    print(f"Column info saved to: {output_path}")

## Next Steps

1. Run `src/preprocessing/feature_engineering.py` to create engineered features
2. Move to Phase 2: Model Building
3. Start with position probability prediction for each horse